In [25]:
import sys
from PIL import Image, ImageDraw

In [26]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action
        

In [27]:
class stackFrontier:
    def __init__(self):
        self.frontier = []
    
    def empty(self):
        return len(self.frontier)==0
    
    def add(self, node):
        return self.frontier.append(node)
    
    def remove(self):
        if self.empty():
            raise Exception("Empty Frotier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node
    
    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

        

In [28]:
class queueFrontier:
    def __init__(self):
        self.frontier = []
    
    def empty(self):
        return len(self.frontier)==0
    
    def remove(self):
        if self.empty():
            return Exception("Empty Frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node
    
    def add(self, node):
        return self.frontier.append(node)
    
    def contains_state(self, state):
        return any(node.state==state for node in self.frontier)

In [29]:
class maze:

    def __init__(self, filename):
        self.filename = filename
    
        with open(filename) as f:
            contents = f.read()
        
        if contents.count("A") != 1:
            raise Exception("Maze must have exactly one start point")
        if contents.count("B") != 1:
            raise Exception("Maze must have exactly one goal")
        
        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j]=="A":
                        self.start = (i,j)
                        row.append(False)
                    elif contents[i][j]=="B":
                        self.goal = (i,j)
                        row.append(False)
                    elif contents[i][j]==" ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)
        
        self.solution = None

    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("█", end="")
                elif (i,j)==self.start:
                    print("A", end="")
                elif (i,j)==self.goal:
                    print("B", end="")
                elif solution is not None and (i,j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()
    
    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up", (row-1, col)),
            ("down", (row+1, col)),
            ("left", (row, col-1)),
            ("right", (row, col+1))
        ]
        result = []
        for action, (r,c) in candidates:
            if 0<=r<self.height and 0<=c<self.width and not self.walls[r][c]:
                result.append((action, (r,c)))
        return result
    
    def solve(self):
        self.num_explored = 0
        start = Node(state=self.start, parent=None, action=None)
        frontier = stackFrontier()
        frontier.add(start)
        self.explored = set()

        while True:
            if frontier.empty():
                raise Exception("No Solution")
            
            node = frontier.remove()
            self.num_explored += 1

            if node.state == self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return
            
            self.explored.add(node.state)

            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)

    def output_image(self, filename, show_solution=True, show_explored=False):
        cell_size = 50
        cell_border = 2

        solution = self.solution[1] if self.solution is not None else None
        explored = self.explored if self.explored is not None else None
        width = self.width
        height = self.height

        img = Image.new(
            "RGBA",
            (width*(cell_size+cell_border), height*(cell_size+cell_border)),
            "black"
        )
        draw = ImageDraw.Draw(img)

        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    fill = (40,40,40)
                elif (i,j)==self.start:
                    fill = (255,0,0)
                elif (i,j)==self.goal:
                    fill = (0,171,28)
                elif solution is not None and show_solution and (i,j) in solution:
                    fill = (220,235,113)
                elif explored is not None and show_explored and (i,j) in explored:
                    fill = (212,97,85)
                else:
                    fill = (237,240,252)
                
                draw.rectangle(
                    ([(j*(cell_size+cell_border), i*(cell_size+cell_border)),
                    ((j+1)*(cell_size+cell_border)-cell_border, (i+1)*(cell_size+cell_border)-cell_border)]),
                    fill=fill
                )
        
        img.save(filename)
    
    def output_image_with_explored(self, filename):
        self.output_image(filename, show_explored=True)

if len(sys.argv) != 2:
    sys.exit("Usage: python maze.py maze.txt")

In [31]:
# Hardcoded filename
maze_file = "maze.txt"

# Create maze object using the hardcoded filename
m = maze(maze_file)

print("File:", maze_file)
print("Maze:")
m.print()
print("Solving...")
m.solve()
print("States Explored:", m.num_explored)
print("Solution:")
m.print()
m.output_image_with_explored("maze.png")

File: maze.txt
Maze:

A████
   ██
 █  █
██ ██
B  ██

Solving...
States Explored: 10
Solution:

A████
***██
 █* █
██*██
B**██

